In [ ]:
# if using Clarifai run this cell every new kaggle session 
# !pip install clarifai-grpc

In [ ]:
# if you want to run the script 
# open it in kaggle & add this dataset to your working directory
# https://www.kaggle.com/datasets/anastasiapetrunia/album-covers-dataset

# to use clarifai`s obj detection you need to get a PAT (Perconal Access Token)
# (https://portal.clarifai.com/signup?recovery_path=%2Fsettings%2Fauthentication) 

In [ ]:
import cv2
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# tf imports
import tensorflow as tf
from tensorflow.keras.models import Model, load_model

# if using Clarifai
# from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
# from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
# from clarifai_grpc.grpc.api.status import status_code_pb2

In [ ]:
# if using Clarifai
def get_predictions_Clarifai(IMAGE_FILE_LOCATION, print_pred = False):
    
    PAT = your_pat_here
    USER_ID = 'task'
    APP_ID = 'my-first-application'

    with open(IMAGE_FILE_LOCATION, "rb") as f:
        file_bytes = f.read()

    request = service_pb2.PostModelOutputsRequest(
        # This is the model ID of a publicly available General model. You may use any other public or custom model ID.
        model_id="general-image-recognition",
        user_app_id=resources_pb2.UserAppIDSet(app_id=APP_ID),
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(image=resources_pb2.Image(base64=file_bytes))
            )
        ],
    )
    response = stub.PostModelOutputs(request, metadata=metadata)

    if response.status.code != status_code_pb2.SUCCESS:
        print(response)
        raise Exception(f"Request failed, status code: {response.status}")
    
    pred = []
    for concept in response.outputs[0].data.concepts:
        name_value = (concept.name, concept.value)
        pred.append(name_value)
        if print_pred:
            print("%12s: %.2f" % name_value)
    return pred

In [ ]:
def detect_expl_content_label(detect_fn, image_np): 
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

#     start = time.time()
    detections = detect_fn(input_tensor)
#     end = time.time()

#     print("The time of execution is :", (end - start) * 10 ** 3, "ms")

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
              	for key, value in detections.items()}
    print(detections['detection_classes'])
    print(detections['detection_scores'])
    if num_detections >= 1:
        boxes = detections['detection_boxes']
        return boxes[0],detections['detection_scores']

    return None

In [ ]:
# if using Clarifai
channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)
PAT = your_pat_here
metadata = (("authorization", f"Key {PAT}"),)

In [ ]:
test_list_path = '../input/album-covers-dataset/test_list.txt'
main_model_path = '../input/album-covers-dataset/resnet101_with_test_list1/resnet101_with_test_list1.h5'
expl_content_model_path = '../input/album-covers-dataset/explicit_content_label_detection/out_export2/saved_model'
images_path = '../input/album-covers-dataset/images labeled/images labeled'
genres = ['rap', 'folk', 'rock', 'disco', 'electro']

In [ ]:
# uploading test files
test_list = []
with open(test_list_path, 'r') as fp:
    for line in fp:
        x = line[:-1]
        test_list.append(x)
    print('Done')

In [ ]:
test_list_sorted = [[],[],[],[],[]]
for x in test_list:
    i = int(x[0])
    test_list_sorted[i].append(x)

In [ ]:
def randomly_select_img(specify_genre,images_path, genre=0):
    if specify_genre:
        img_name = random.choice(test_list_sorted[genre])
    else:
        img_name = random.choice(test_list)
        genre = int(img_name[0])
    img_path = images_path+'/'+genres[genre]+'/'+img_name
    return img_path

In [ ]:
img_path = randomly_select_img(False,images_path) 
# img_path = '../input/album-covers-dataset/images labeled/images labeled/rap/0_491.jpg'
# load image
img_arr=cv2.imread(img_path)
img_arr=cv2.resize(img_arr,(224,224))
img_data = np.array([img_arr])
plt.imshow(img_arr)
plt.show()

In [ ]:
img_path

Main Model

In [ ]:
main_model = load_model(main_model_path)

In [ ]:
pred = main_model.predict(img_data)
for y in pred[0]:
    print(y)
genre=np.where(pred[0] == max(pred[0]))
print(genres[genre[0][0]])

Explicit Content Label Detection

In [ ]:
# load the model
detect_fn = tf.saved_model.load(expl_content_model_path)

In [ ]:
# load image
image = Image.open(img_path)
image_data = np.array(image)
#image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB) # needed only for PNG images
(height, width, channels) = image_data.shape

boxes,score = detect_expl_content_label(detect_fn, image_data)
if boxes is not None:
    x1 = int(boxes[1] * width)
    y1 = int(boxes[0] * height)
    x2 = int(boxes[3] * width)
    y2 = int(boxes[2] * height)
#     print("Box: [%d, %d]-[%d, %d]" % (x1, y1, x2, y2))

In [ ]:
if score > 0.4:
    img = cv2.imread(img_path)
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0),thickness=5)
    plt.imshow(img)
    plt.show()
else:
    print('Not detected')

Clarifai Detection

In [ ]:
pred = get_predictions_Clarifai(img_path)

In [ ]:
pred